# Introduction
## create an adder circuit

In [1]:
from qiskit import QuantumCircuit
from qiskit.providers.aer import AerSimulator

#make new simulator
sim = AerSimulator()

# create 4 qubits and 2 classical bits
test_qc = QuantumCircuit(4, 2)

# encode an input
test_qc.x(0)
test_qc.x(1)

# create the adder circuit
test_qc.cx(0,2)
test_qc.cx(1,2)
test_qc.ccx(0,1,3)

# measure the bottom two qubits to extract the output
test_qc.measure(2,0)
test_qc.measure(3,1)
test_qc.draw()

┌───┐                     
q_0: ┤ X ├──■─────────■────────
     ├───┤  │         │        
q_1: ┤ X ├──┼────■────■────────
     └───┘┌─┴─┐┌─┴─┐  │  ┌─┐   
q_2: ─────┤ X ├┤ X ├──┼──┤M├───
          └───┘└───┘┌─┴─┐└╥┘┌─┐
q_3: ───────────────┤ X ├─╫─┤M├
                    └───┘ ║ └╥┘
c: 2/═════════════════════╩══╩═
                          0  1

In [2]:
job = sim.run(test_qc)
result = job.result()
result.get_counts()

{'10': 1024}

## State vectors
$$\left | x \right \rangle=
 \left[
 \begin{matrix}
   \sqrt{\frac{1}{2}} \\
   \sqrt{\frac{1}{2}}\\
   0 \\
   0 \\
  \end{matrix}
  \right]
$$
这样的向量表示00态振幅为$\sqrt{\frac{1}{2}}$概率为1/2,01态同理。
$$\left | 00 \right \rangle=
 \left[
 \begin{matrix}
   1 \\
   0\\
   0 \\
   0 \\
  \end{matrix}
  \right]，
  \left | 01 \right \rangle=
 \left[
 \begin{matrix}
   0 \\
   1 \\
   0 \\
   0 \\
  \end{matrix}
  \right]
$$
$$ \left | x \right \rangle = \sqrt{\frac{1}{2}}(\left | 00 \right \rangle + \left | 01 \right \rangle)$$
$\left | 00 \right \rangle$和$\left | 01 \right \rangle$是$\left | x \right \rangle$的两个基矢，$\left | x \right \rangle$在两个基矢上的投影都是$\sqrt{\frac{1}{2}}$
$$\left | + \right \rangle = 
\sqrt{\frac{1}{2}} \left[
\begin{matrix}
  1 \\
  1 \\
\end{matrix}
\right],
\left | - \right \rangle = 
\sqrt{\frac{1}{2}} \left[
\begin{matrix}
  1 \\
  -1 \\
\end{matrix}
\right]
$$


## operations

 $$CNOT = \left[
 \begin{matrix}
   1 & 0 & 0 & 0 \\
   0 & 0 & 0 & 1\\
   0 & 0 & 1 & 0 \\
   0 & 1 & 0 & 0 \\
  \end{matrix}
  \right],
  H = \sqrt{\frac{1}{2}}\left[
  \begin{matrix}
  1 & 1 \\
   1 & -1 \\
  \end{matrix}
  \right]$$

## entangled states
The entangled states couldn't simply discribe by two single qubit states
$$\left | \Phi^+ \right \rangle=
 \sqrt{\frac{1}{2}}\left[
 \begin{matrix}
   1 \\
   0\\
   0 \\
   1 \\
  \end{matrix}
  \right]$$

## Entangling gates

create $\left | \Phi^+ \right \rangle$

In [3]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

qc = QuantumCircuit(2)

qc.h(1)
qc.cx(1, 0) # control is qubit1
qc.draw()

┌───┐
q_0: ─────┤ X ├
     ┌───┐└─┬─┘
q_1: ┤ H ├──■──
     └───┘

In [4]:
ket = Statevector(qc)
ket.draw()

'Statevector([0.70710678+0.j, 0.        +0.j, 0.        +0.j,\n             0.70710678+0.j],\n            dims=(2, 2))'

## The power of entanglement
### why the entanglement powerful

we need to keep track up $2^n$ amplitudes to fully describe the state of n qubits. if they don't be entangled what means each qubits is indepent and we just need $2n$ amplitudes to discribe them.

### example
we can use one qubit to sent two bit message, suppose we have two people: Alice and Bob.Alice wishes to send a two bit message to Bob and wants to do it by sending qubits.And we imagine a third party charlie, whose job is just to create a entangled state and send the qubits out.

In [5]:
qc_charlie = QuantumCircuit(2,2)

qc_charlie.h(1)
qc_charlie.cx(1,0)

qc_charlie.draw()

┌───┐
q_0: ─────┤ X ├
     ┌───┐└─┬─┘
q_1: ┤ H ├──■──
     └───┘     
c: 2/══════════

In [6]:
MESSAGE = '01'

qc_alice = QuantumCircuit(2,2)

if MESSAGE[-2] == '1':
    qc_alice.z(1)
if MESSAGE[-1] == '1':
    qc_alice.x(1)

In [7]:
qc_bob = QuantumCircuit(2,2)

qc_bob.cx(1,0)
qc_bob.h(1)

qc_bob.measure([0,1],[0,1])

qc_bob.draw()

┌───┐     ┌─┐   
q_0: ┤ X ├─────┤M├───
     └─┬─┘┌───┐└╥┘┌─┐
q_1: ──■──┤ H ├─╫─┤M├
          └───┘ ║ └╥┘
c: 2/═══════════╩══╩═
                0  1

In [9]:
# simulate
from qiskit import Aer
backend = Aer.get_backend('aer_simulator')

# compose the circuit each one create
complete_qc = qc_charlie.compose(qc_alice.compose(qc_bob))
backend.run(complete_qc).result().get_counts()

{'01': 1024}

in all, alice could use z or x state at one qubit to get four entangled states which makes alice send two bits' maeeage with just one qubit.

## The Hardy paradox
  There are limits on the kind of correlations possible with classical system.Wecan apply chains of logic, which use the results of ane to deduce things about other.
  
  But for qubits, the two features is not independent, there is the uncertainty principle.We using $\left | 0 \right \rangle$ and $\left | 1 \right \rangle $ states to encode the bit values 0 and 1 and measured by z measurement(measure directly), using $\left | + \right \rangle$ and $\left | - \right \rangle $ states to encode and measured by x measurement(using H gate and then measure).We define a quantity$\left \langle Z \right \rangle  = p_z(0) - p_z(1)$,$p_z(0)$ is the probability of 0.There is$$\left \langle Z \right \rangle^2 + \left \langle X \right \rangle^2 \le 1$$
  which different from classical is that we use different gates on qubit to measure different feature.

## Visualizing Entanglement

In [18]:
from qiskit_textbook.games import hello_quantum
exercises = {
    'initialize': [],
    'success_condition': {},
    'allowed_gates': {'0': {'x': 3}, '1': {}, 'both': {}},
    'vi': [[1], True, False],
    'mode': 'line',
    'qubit_names': {'0':'q[0]', '1':'q[1]'}
}
def run_puzzle():
    puzzle = hello_quantum.run_game(exercises['initialize'],
                           exercises['success_condition'],
                           exercises['allowed_gates'],
                           exercises['vi'],
                           qubit_names=exercises['qubit_names'],
                           mode=exercises['mode']
                          )
    return puzzle

In [20]:
puzzle = run_puzzle()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x06\x00\x00\x00\xbef\x98\xdc\x…

For I don't have python file hello_qiskit.py in my environment, and I couldn't find it . so this chapter will not have code anymore.

The vertical one represents standard measurements (also known as 'z measurements'), and the horizontal one represents the 'x measurements' 

## Bell's Inequalities

In [1]:
import random
def setup_variables():
    
    ### Replace this section with anything you want ###
    
    r = random.random()
    
    A = r*(2/3)
    B = r*(1/3)
    
    ### End of section ###
    
    return A, B


def hash2bit(variable, hash_type):
    
    ### Replace this section with anything you want ###
    
    if hash_type == 'V':
        bit = (variable < 0.5)
    elif hash_type == 'H':
        bit = (variable < 0.25)
        
    bit = str(int(bit)) # Turn True or False into '1' and '0'
    
    ### End of section ###
        
    return bit

shots = 8192
def calculate_P():
    P = {}
    for hashes in ['VV','VH','HV','HH']:
        
        # calculate each P[hashes] by sampling over `shots` samples
        P[hashes] = 0
        for shot in range(shots):

            A, B = setup_variables()

            # hash type for variable `A` is the 1st character of `hashes`
            a = hash2bit(A, hashes[0])
            # hash type for variable `B` is the 2nd character of `hashes`
            b = hash2bit(B, hashes[1])

            P[hashes] += (a != b)/shots
 
    return P

P = calculate_P()
print(P)

{'VV': 0.2506103515625, 'VH': 0.0, 'HV': 0.615966796875, 'HH': 0.369384765625}


There is $$P['HH'] \le P['HV'] + P['VH'] + P['VV']$$
For qubits, it's quite different.One reason is that we don't know the result of hashes before we actually do it. The other reason is that the results is contextual. So we can't make assumption like classical.

## Grover's search algorithm

所有的解构成一个向量
$$\left | \surd  \right \rangle  = \frac{1}{\sqrt{3}}(\left | 000 \right \rangle + \left | 011 \right \rangle + \left | 101 \right \rangle)$$
其他情况构成另一个向量
$$\left | X \right \rangle  = \frac{1}{\sqrt{3}}(\left | 001 \right \rangle + \left | 010 \right \rangle + \left | 100 \right \rangle + \left | 110 \right \rangle + \left | 111 \right \rangle)$$
我们给定一个初始状态$\left | s \right \rangle$,该初始状态表示所有可能情况，
在解为解空间很小一部分时，$\left | s \right \rangle$非常接近$\left | X \right \rangle$,接下来根据$\left | X \right \rangle$对$\left | s \right \rangle$进行翻转，再将翻转的向量根据$\left | s \right \rangle$进行翻转，重复直到所得的向量接近$\left | \surd \right \rangle$。

在这里初始$\left | s \right \rangle$与$\left | X \right \rangle$的夹角是$\frac{1}{\sqrt{N}}$, 此后每次翻转增加的角度为$\frac{2}{\sqrt{N}}$,而$\left | \surd  \right \rangle$与$\left | X \right \rangle$向量相互垂直。

下面是电路实现

In [9]:
from qiskit import QuantumCircuit

oracle = QuantumCircuit(2)
oracle.cz(0, 1) # the solve is 11, in will only flips state 11

diffuser = QuantumCircuit(2)
diffuser.h([0, 1])
diffuser.x([0, 1])
diffuser.cz(0,1)
diffuser.x([0,1])
diffuser.h([0,1])

# create s
grover = QuantumCircuit(2)
grover.h([0, 1])

grover = grover.compose(oracle)
grover = grover.compose(diffuser)
grover.measure_all()
grover.draw()

┌───┐   ┌───┐┌───┐   ┌───┐┌───┐ ░ ┌─┐   
   q_0: ┤ H ├─■─┤ H ├┤ X ├─■─┤ X ├┤ H ├─░─┤M├───
        ├───┤ │ ├───┤├───┤ │ ├───┤├───┤ ░ └╥┘┌─┐
   q_1: ┤ H ├─■─┤ H ├┤ X ├─■─┤ X ├┤ H ├─░──╫─┤M├
        └───┘   └───┘└───┘   └───┘└───┘ ░  ║ └╥┘
meas: 2/═══════════════════════════════════╩══╩═
                                           0  1

In [10]:
from qiskit import Aer
sim = Aer.get_backend('aer_simulator')
sim.run(grover).result().get_counts()

{'11': 1024}

## example
find the solution of ${\textstyle \bigwedge_{\neg  {\textstyle \bigwedge_{\neg x}^{\neg y}} }^{\neg y}} $

In [8]:
from qiskit import QuantumCircuit


oracle = QuantumCircuit(5,2)
oracle.x([0,1])
oracle.ccx(0,1,2)
oracle.x([0,1])

oracle.x(1)
oracle.cx(1,3)
oracle.x(1)

oracle.x(2)
oracle.ccx(2,3,4)
oracle.x(2)

oracle.x(1)
oracle.cx(1,3)
oracle.x(1)

oracle.x([0,1])
oracle.ccx(0,1,2)
oracle.x([0,1])


diffuser = QuantumCircuit(5,2)

diffuser.h([0, 1])
diffuser.x([0, 1])
diffuser.ccx(0,1,4)
diffuser.x([0,1])
diffuser.h([0,1])


grover = QuantumCircuit(5,2)
grover.h([0, 1])

grover = grover.compose(oracle)
grover = grover.compose(diffuser)
grover.measure([0,1],[0,1])
grover.draw()

┌───┐┌───┐     ┌───┐┌───┐                                   ┌───┐┌───┐»
q_0: ┤ H ├┤ X ├──■──┤ X ├┤ X ├────────────────────────────────■──┤ X ├┤ H ├»
     ├───┤├───┤  │  ├───┤├───┤     ┌───┐┌───┐     ┌───┐┌───┐  │  ├───┤├───┤»
q_1: ┤ H ├┤ X ├──■──┤ X ├┤ X ├──■──┤ X ├┤ X ├──■──┤ X ├┤ X ├──■──┤ X ├┤ H ├»
     └───┘└───┘┌─┴─┐├───┤└───┘  │  └───┘├───┤  │  └───┘└───┘┌─┴─┐└───┘└───┘»
q_2: ──────────┤ X ├┤ X ├───────┼────■──┤ X ├──┼────────────┤ X ├──────────»
               └───┘└───┘     ┌─┴─┐  │  └───┘┌─┴─┐          └───┘          »
q_3: ─────────────────────────┤ X ├──■───────┤ X ├─────────────────────────»
                              └───┘┌─┴─┐     └───┘                         »
q_4: ──────────────────────────────┤ X ├───────────────────────────────────»
                                   └───┘                                   »
c: 2/══════════════════════════════════════════════════════════════════════»
                                                                           »
«     ┌───┐     ┌───┐┌───┐┌─┐   
«q_0: ┤ X ├──■──┤ X ├┤ H ├┤M├───
«     ├───┤  │  ├───┤├───┤└╥┘┌─┐
«q_1: ┤ X ├──■──┤ X ├┤ H ├─╫─┤M├
«     └───┘  │  └───┘└───┘ ║ └╥┘
«q_2: ───────┼─────────────╫──╫─
«            │             ║  ║ 
«q_3: ───────┼─────────────╫──╫─
«          ┌─┴─┐           ║  ║ 
«q_4: ─────┤ X ├───────────╫──╫─
«          └───┘           ║  ║ 
«c: 2/═════════════════════╩══╩═
«                          0  1

In [9]:
from qiskit import Aer
sim = Aer.get_backend('aer_simulator')
sim.run(grover).result().get_counts()

{'01': 614, '00': 147, '11': 138, '10': 125}